# Import Datasets

In [1]:
import pandas as pd
import numpy as np
import json

data = pd.read_csv("Mass Shootings Dataset Ver 5.csv",encoding="ISO-8859-1")
data

,S#,Title,Location,Date,Incident Area,Open/Close Location,Target,Cause,Summary,Fatalities,Injured,Total victims,Policeman Killed,Age,Employeed (Y/N),Employed at,Mental Health Issues,Race,Gender,Latitude,Longitude
0,1,Texas church mass shooting,"Sutherland Springs, TX",11/5/2017,Church,Close,random,unknown,"Devin Patrick Kelley, 26, an ex-air force offi...",26,20,46,0.0,26,NaN,NaN,No,White,M,NaN,NaN
1,2,Walmart shooting in suburban Denver,"Thornton, CO",11/1/2017,Wal-Mart,Open,random,unknown,"Scott Allen Ostrem, 47, walked into a Walmart ...",3,0,3,0.0,47,NaN,NaN,No,White,M,NaN,NaN
2,3,Edgewood businees park shooting,"Edgewood, MD",10/18/2017,Remodeling Store,Close,coworkers,unknown,"Radee Labeeb Prince, 37, fatally shot three pe...",3,3,6,0.0,37,NaN,Advance Granite Store,No,Black,M,NaN,NaN
3,4,Las Vegas Strip mass shooting,"Las Vegas, NV",10/1/2017,Las Vegas Strip Concert outside Mandala Bay,Open,random,unknown,"Stephen Craig Paddock, opened fire from the 32...",59,527,585,1.0,64,NaN,NaN,Unclear,White,M,36.181271,-115.134132
4,5,San Francisco UPS shooting,"San Francisco, CA",6/14/2017,UPS facility,Close,coworkers,NaN,"Jimmy Lam, 38, fatally shot three coworkers an...",3,2,5,0.0,38,1.0,NaN,Yes,Asian,M,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,319,Clara Barton Elementary School,"Chicago, Illinois",1/17/1974,Clara Barton Elementary School,Close,Teachers,anger,"On January 17, 1974, a 14-year-old student ent...",1,3,4,0.0,14,NaN,NaN,Yes,Unknown,Male,41.839280,-87.688181
319,320,New Orleans Police Shootings,"New Orleans, Louisiana",12/31/1972,NaN,NaN,random,psycho,"On New Year's Eve in 1972, a 23-year-old ex-Na...",10,13,22,4.0,23,NaN,NaN,Yes,Black American or African American,Male,30.068724,-89.931474
320,321,St. Aloysius Church,"Spokane, Washington",11/11/1971,Church,Close,random,terrorism,"On November 11, 1971, a former MIT student ent...",2,4,5,0.0,NaN,NaN,NaN,Yes,White American or European American,Male,47.673674,-117.415984
321,322,Rose-Mar College of Beauty,"Mesa, Arizona",11/12/1966,Rose-Mar College of Beauty,Close,random,terrorism,"On November 12, 1966, an 18-year-old high scho...",5,1,6,0.0,18,NaN,NaN,Yes,White American or European American,Male,33.422687,-111.816320


# Encoding the Race

In [2]:
data['Race']= data['Race'].fillna("Unknown")
data['Race'].value_counts()

White American or European American                    122
Black American or African American                      76
Unknown                                                 44
Some other race                                         20
white                                                   12
Asian American                                          11
White                                                    9
Asian                                                    6
Black                                                    5
Latino                                                   5
Native American or Alaska Native                         3
black                                                    3
Other                                                    2
Two or more races                                        2
Black American or African American/Unknown               1
Asian American/Some other race                           1
White American or European American/Some other Race     

In [3]:
def race_encode(df):
    if("white" in df['Race'].lower()):
        return "White"
    elif("black" in df['Race'].lower()):
        return "Black"
    elif('asian' in df['Race'].lower()):
        return "Asian"
    elif("uknown" in df['Race'].lower()):
        return "Unknown"
    else:
        return "Other"

data['Race_encoded']= data.apply(race_encode,axis=1)
data.head()

,S#,Title,Location,Date,Incident Area,Open/Close Location,Target,Cause,Summary,Fatalities,Injured,Total victims,Policeman Killed,Age,Employeed (Y/N),Employed at,Mental Health Issues,Race,Gender,Latitude,Longitude,Race_encoded
0,1,Texas church mass shooting,"Sutherland Springs, TX",11/5/2017,Church,Close,random,unknown,"Devin Patrick Kelley, 26, an ex-air force offi...",26,20,46,0.0,26,NaN,NaN,No,White,M,NaN,NaN,White
1,2,Walmart shooting in suburban Denver,"Thornton, CO",11/1/2017,Wal-Mart,Open,random,unknown,"Scott Allen Ostrem, 47, walked into a Walmart ...",3,0,3,0.0,47,NaN,NaN,No,White,M,NaN,NaN,White
2,3,Edgewood businees park shooting,"Edgewood, MD",10/18/2017,Remodeling Store,Close,coworkers,unknown,"Radee Labeeb Prince, 37, fatally shot three pe...",3,3,6,0.0,37,NaN,Advance Granite Store,No,Black,M,NaN,NaN,Black
3,4,Las Vegas Strip mass shooting,"Las Vegas, NV",10/1/2017,Las Vegas Strip Concert outside Mandala Bay,Open,random,unknown,"Stephen Craig Paddock, opened fire from the 32...",59,527,585,1.0,64,NaN,NaN,Unclear,White,M,36.181271,-115.134132,White
4,5,San Francisco UPS shooting,"San Francisco, CA",6/14/2017,UPS facility,Close,coworkers,NaN,"Jimmy Lam, 38, fatally shot three coworkers an...",3,2,5,0.0,38,1.0,NaN,Yes,Asian,M,NaN,NaN,Asian


In [4]:
data['Race_encoded'].value_counts()

White    144
Black     85
Other     76
Asian     18
Name: Race_encoded, dtype: int64

# Determining State using Co-ordinates

In [5]:
!pip install reverse_geocoder
import reverse_geocoder as rg

In [6]:
data= data.dropna(subset = ['Latitude','Longitude'])
lat_lst = data['Latitude'].to_list()
long_lst = data['Longitude'].to_list()
x=[]
for i in range(len(lat_lst)):
    y = tuple((lat_lst[i],long_lst[i]))
    x.append(y)
data['Coordinates']= x
result = rg.search(x)
state_lst= []
for i in range(len(result)):
    state_lst.append(result[i]['admin1'].upper())
data['State'] = state_lst
data

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Loading formatted geocoded file...


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


,S#,Title,Location,Date,Incident Area,Open/Close Location,Target,Cause,Summary,Fatalities,Injured,Total victims,Policeman Killed,Age,Employeed (Y/N),Employed at,Mental Health Issues,Race,Gender,Latitude,Longitude,Race_encoded,Coordinates,State
3,4,Las Vegas Strip mass shooting,"Las Vegas, NV",10/1/2017,Las Vegas Strip Concert outside Mandala Bay,Open,random,unknown,"Stephen Craig Paddock, opened fire from the 32...",59,527,585,1.0,64,NaN,NaN,Unclear,White,M,36.181271,-115.134132,White,"(36.181271, -115.13413200000001)",NEVADA
14,15,"Ferguson, MO Drive by","Ferguson, Missouri",4/29/2016,NaN,Open,random,NaN,A group of 15 to 20 people was gathered for a ...,0,4,4,0.0,0,NaN,NaN,Unknown,Unknown,Unknown,38.744217,-90.305391,Other,"(38.744217, -90.305391)",MISSOURI
15,16,"Forestville, Maryland Drive-by",NaN,4/26/2016,in street,Open,NaN,NaN,Shooter shot from his car at people standing o...,1,4,5,0.0,NaN,NaN,NaN,Unknown,Unknown,Unknown,38.845113,-76.874972,Other,"(38.845113, -76.874972)",MARYLAND
16,17,"Halifax County, VA",NaN,4/24/2016,crown,Open,random,NaN,Male shooter fired into crown when a deputy po...,0,6,6,0.0,NaN,NaN,NaN,Unknown,Black American or African American,Male,36.765971,-78.928344,Black,"(36.765971, -78.928344)",VIRGINIA
17,18,Tire-Slashing revenge escalation,NaN,4/21/2016,block party,Open,random,frustration,Shooter was angry over fact that rival familie...,0,4,4,0.0,NaN,NaN,NaN,Unknown,Black American or African American,Male,39.290385,-76.612189,Black,"(39.290385, -76.612189)",MARYLAND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,319,Clara Barton Elementary School,"Chicago, Illinois",1/17/1974,Clara Barton Elementary School,Close,Teachers,anger,"On January 17, 1974, a 14-year-old student ent...",1,3,4,0.0,14,NaN,NaN,Yes,Unknown,Male,41.839280,-87.688181,Other,"(41.83928045, -87.68818145)",ILLINOIS
319,320,New Orleans Police Shootings,"New Orleans, Louisiana",12/31/1972,NaN,NaN,random,psycho,"On New Year's Eve in 1972, a 23-year-old ex-Na...",10,13,22,4.0,23,NaN,NaN,Yes,Black American or African American,Male,30.068724,-89.931474,Black,"(30.0687242, -89.93147412)",LOUISIANA
320,321,St. Aloysius Church,"Spokane, Washington",11/11/1971,Church,Close,random,terrorism,"On November 11, 1971, a former MIT student ent...",2,4,5,0.0,NaN,NaN,NaN,Yes,White American or European American,Male,47.673674,-117.415984,White,"(47.67367375, -117.41598429999999)",WASHINGTON
321,322,Rose-Mar College of Beauty,"Mesa, Arizona",11/12/1966,Rose-Mar College of Beauty,Close,random,terrorism,"On November 12, 1966, an 18-year-old high scho...",5,1,6,0.0,18,NaN,NaN,Yes,White American or European American,Male,33.422687,-111.816320,White,"(33.42268696, -111.8163202)",ARIZONA


# Name-entity Recognition (for Word Cloud)

In [7]:
!pip install flair

In [8]:
output = data.to_json(orient="records")
output = json.loads(output)

In [9]:
from flair.data import Sentence
from flair.models import SequenceTagger

# load tagger
tagger = SequenceTagger.load("flair/ner-english-large")

# print predicted NER spans
print('The following NER tags are found:')

# iterate over entities and print
count = 0
for s in data['Summary']:
    # make example sentence
    sentence = Sentence(s)
    # predict NER tags
    tagger.predict(sentence)
    # get the results
    results = sentence.to_dict(tag_type='ner')["entities"]
    temp = []
    for r in results:
        temp.append({"text": r["text"], "labels": r["labels"][0].value})
    print(temp)
    output[count]["NER"] = temp
    count += 1



2021-11-24 07:48:22,338 loading file /root/.flair/models/ner-english-large/07301f59bb8cb113803be316267f06ddf9243cdbba92a4c8067ef92442d2c574.554244d3476d97501a766a98078421817b14654496b86f2f7bd139dc502a4f29
The following NER tags are found:
[{'text': 'Stephen Craig Paddock', 'labels': 'PER'}, {'text': 'Manadalay Bay', 'labels': 'LOC'}, {'text': 'Last Vegas', 'labels': 'LOC'}]
[]
[]
[]
[]
[]
[]
[{'text': 'Alabama', 'labels': 'LOC'}]
[]
[]
[]
[{'text': 'South Shore', 'labels': 'LOC'}, {'text': 'Chicago', 'labels': 'LOC'}]
[]
[{'text': 'Virginia', 'labels': 'LOC'}, {'text': 'Greyhound', 'labels': 'LOC'}]
[]
[]
[{'text': 'Alabama', 'labels': 'LOC'}]
[]
[{'text': 'Iraq', 'labels': 'LOC'}]
[{'text': 'Spring Break', 'labels': 'MISC'}]
[{'text': 'Wetumpka', 'labels': 'LOC'}, {'text': 'Alabama', 'labels': 'LOC'}]
[]
[]
[]
[{'text': 'Trenton', 'labels': 'LOC'}, {'text': 'NJ', 'labels': 'LOC'}, {'text': 'Acura', 'labels': 'ORG'}]
[]
[]
[]
[{'text': 'Kansas City', 'labels': 'LOC'}, {'text': 'Kansas'

In [10]:
with open('mass_shootings.json', 'w') as json_file:
    json.dump(output, json_file, indent=4)